# Introduction

This notebook will include experimental results on the Actor-Critic agent specified in the RL book by Rich Sutton on the smart vacuum environment.

In [1]:
# import needed libs

%load_ext autoreload

# Auto reloading causes the kernel to reload the libraries we have
%autoreload 2

# usual imports for visualization, etc.
import numpy as np
import matplotlib.pyplot as plt

import datetime

# make it reproducible
np.random.seed(0)

# show plots inline
%matplotlib inline

In [ ]:
# Some initializations

from envs import SmartVac
from agents import ActorCriticAgent

max_episode_steps = 10000
results_folder = 'res/'
figs_folder = 'figs/'